# QueryPolicy Examples

# Import to run %sh magic cell

In [14]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

# Add Java Client POM Dependency

In [15]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>5.1.11</version>
  </dependency>
</dependencies>

# Add required Imports

In [16]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


# Generate Test Data

In [17]:
%sh aql -c "truncate test"

In [18]:
%sh aql -c "set key_send true"

In [19]:
%sh aql -f "./aqlScripts/insert.aql"

In [20]:
AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Key key = new Key("test", "testset", "key1");
System.out.println("Working with record key:");
System.out.println(key);

Record record = client.get(null, key);
System.out.println("Read back the record.");

System.out.println("Record values are:");
System.out.println(record);

Initialized the client and connected to the cluster.
Working with record key:
test:testset:key1:bf6c1d13e7cd10c5bd022d27e7df170c0bccd6e1
Read back the record.
Record values are:
(gen:1),(exp:409884015),(bins:(name:Jack),(age:26))


In [21]:
%sh aql -c "create index idx_age on test.testset (age) numeric"

In [23]:
//Needed imports
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.Operation;
import com.aerospike.client.Bin;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.query.RecordSet;

//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
stmt.setFilter(Filter.range("age", 20,60));
//Operation[] ops = new Operation[1];
//Bin ageBinNull = Bin.asNull("age");
//ops[0] = Operation.put(ageBinNull);
//stmt.setOperations(ops);
QueryPolicy qp = new QueryPolicy();
qp.recordQueueSize = 5000;
RecordSet rs = client.query(qp, stmt);

int nCount = 0;
while(rs.next()){
    Record r = rs.getRecord();
    Key k = rs.getKey();  
    System.out.println(r);
    System.out.println(k);
    Bin newBin = new Bin("newBin", nCount);
    nCount++;
    client.put(null, k, newBin);
    System.out.println(client.get(null,k));
}


(gen:2),(exp:409884018),(bins:(name:Jim),(age:46),(newBin:4))
test:testset:key4:703ca4a03068c57171bc4be1f829fcdd604ccdc5
(gen:3),(exp:409884070),(bins:(name:Jim),(age:46),(newBin:0))
(gen:2),(exp:409884018),(bins:(name:Jack),(age:26),(newBin:5))
test:testset:key1:bf6c1d13e7cd10c5bd022d27e7df170c0bccd6e1
(gen:3),(exp:409884070),(bins:(name:Jack),(age:26),(newBin:1))
(gen:2),(exp:409884018),(bins:(name:James),(age:38),(newBin:6))
test:testset:key3:10fd8f59adf1833152e439a2e03c19efcb12c145
(gen:3),(exp:409884070),(bins:(name:James),(age:38),(newBin:2))
(gen:2),(exp:409884018),(bins:(name:Susan),(age:42),(newBin:0))
test:testset:key9:cf4f71a9d9f5e73ffc640e387c77c0a3b98e1164
(gen:3),(exp:409884070),(bins:(name:Susan),(age:42),(newBin:3))
(gen:2),(exp:409884018),(bins:(name:Sean),(age:24),(newBin:7))
test:testset:key7:1ab5bb58a1d1e17dcf398a8039ddd09f7460606a
(gen:3),(exp:409884070),(bins:(name:Sean),(age:24),(newBin:4))
(gen:2),(exp:409884018),(bins:(name:Sally),(age:32),(newBin:1))
test:test